In [ ]:
! pip3 install transformers

In [ ]:
! pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [13]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [14]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [15]:
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [17]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [20]:
sample = '/content/common_voice_mr_27591986.wav'
result = pipe(sample)
print(result["text"])

 या पानास लेखासे स्वरूप याईला हवे।


In [21]:
reference = "या पानास लेखाचे स्वरूप यायला हवे"

In [22]:
def calculate_wer(reference, hypothesis):
	ref_words = reference.split()
	hyp_words = hypothesis.split()
	# Counting the number of substitutions, deletions, and insertions
	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
	deletions = len(ref_words) - len(hyp_words)
	insertions = len(hyp_words) - len(ref_words)
	# Total number of words in the reference text
	total_words = len(ref_words)
	# Calculating the Word Error Rate (WER)
	wer = (substitutions + deletions + insertions) / total_words
	return wer

In [23]:
print(calculate_wer(reference,result["text"]))

0.5


In [77]:
sample = '/content/common_voice_mr_27593327.wav'
result = pipe(sample)
print(result["text"])

 कुटिन्नो क्रिश्चन आणि त्यातही गुव्यातले त्यामले त्यांचा कडे परत असल्याचा इंग्रजानना साउशेही आला नाही।


In [78]:
reference = "कुटिन्हो ख्रिश्चन आणि त्यातही गोव्यातले त्यामुळे त्यांच्याकडे प्रत असल्याचा इंग्रजांना संशयही आला नाही"

In [81]:
def calculate_wer(reference, hypothesis):
	ref_words = reference.split()
	hyp_words = hypothesis.split()
	# Counting the number of substitutions, deletions, and insertions
	substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
	deletions = len(ref_words) - len(hyp_words)
	insertions = len(hyp_words) - len(ref_words)
	# Total number of words in the reference text
	total_words = len(ref_words)
	# Calculating the Word Error Rate (WER)
	wer = (substitutions + deletions + insertions) / total_words
	return wer

In [83]:
from jiwer import wer

# Calculate WER for each pair of reference and hypothesis
error_rate = wer(reference, hypothesis)
print(f"WER: {error_rate:.2%}")

WER: 100.00%


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
import os
folder_path = "/content/drive/MyDrive/Sample_voice_test"
for filename in os.listdir(folder_path):
    if filename.endswith((".wav", ".mp3", ".flac")):
        audio_path = os.path.join(folder_path, filename)
        print(audio_path)

/content/drive/MyDrive/Sample_voice_test/common_voice_mr_27591990.wav
/content/drive/MyDrive/Sample_voice_test/common_voice_mr_27591988.wav
/content/drive/MyDrive/Sample_voice_test/common_voice_mr_27591999.wav
/content/drive/MyDrive/Sample_voice_test/common_voice_mr_27591986.wav


In [61]:
import os
folder_path = "/content/drive/MyDrive/Sample_voice_test"
for filename in os.listdir(folder_path):
    if filename.endswith((".wav", ".mp3", ".flac")):
        audio_path = os.path.join(folder_path, filename)
        sample = audio_path
        result = pipe(sample)
        print(f"Transcription for {filename}: {result['text']}")

Transcription for common_voice_mr_27591990.wav:  संत न्यानेश्वरांच्या सात्व्या जन्म स्यताब्दि निमित्त तानी सर्वप्रथम मोग्रा फुललाया कादंबरी से अभिवाचन केले उते।
Transcription for common_voice_mr_27591988.wav:  सत्यवती अम्बालिकेला व्यासाना बगितल्यावर डोए मिटू नोकोस असे सांगते।
Transcription for common_voice_mr_27591999.wav:  पंजाब एक रुषी प्रदान राज्या है।
Transcription for common_voice_mr_27591986.wav:  या पानास लेखासे स्वरूप याईला हवे।


In [62]:
import os
folder_path = "/content/drive/MyDrive/Sample_voice_test"

transcriptions = []

for filename in os.listdir(folder_path):
    if filename.endswith((".wav", ".mp3", ".flac")):
        audio_path = os.path.join(folder_path, filename)
        sample = audio_path
        result = pipe(sample)
        transcription = result["text"]
        transcriptions.append(transcription)
print(transcriptions)

[' संत न्यानेश्वरांच्या सात्व्या जन्म स्यताब्दि निमित्त तानी सर्वप्रथम मोग्रा फुललाया कादंबरी से अभिवाचन केले उते।', ' सत्यवती अम्बालिकेला व्यासाना बगितल्यावर डोए मिटू नोकोस असे सांगते।', ' पंजाब एक रुषी प्रदान राज्या है।', ' या पानास लेखासे स्वरूप याईला हवे।']


In [63]:
hypothesis_texts = [
    "संत ज्ञानेश्वरांच्या सातव्या जन्मशताब्दीनिमित्त त्यांनी सर्वप्रथम मोगरा फुलला या कादंबरीचे अभिवाचन केले होते",
    "सत्यवती अंबालिकेला व्यासांना बघितल्यावर डोळे मिटू नकोस असे सांगते",
    "पंजाब एक कृषि प्रधान राज्य आहे"
    "या पानास लेखाचे स्वरूप यायला हवे"
]

In [69]:
for reference, hypothesis in zip(transcriptions, hypothesis_texts):
  ref_words = reference.split()
  hyp_words = hypothesis.split()
  # Counting the number of substitutions, deletions, and insertions
  substitutions = sum(1 for ref, hyp in zip(ref_words, hyp_words) if ref != hyp)
  deletions = len(ref_words) - len(hyp_words)
  insertions = len(hyp_words) - len(ref_words)
  # Total number of words in the reference text
  total_words = len(ref_words)
  # Calculating the Word Error Rate (WER)
  wer = (substitutions + deletions + insertions) / total_words
  print(wer)

0.8
0.6666666666666666
0.6666666666666666


In [66]:
! pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.4 MB/s eta 0:00:00


In [68]:
from jiwer import wer

# Calculate WER for each pair of reference and hypothesis
for reference, hypothesis in zip(transcriptions, hypothesis_texts):
    error_rate = wer(reference, hypothesis)
    print(f"WER: {error_rate:.2%}")

WER: 73.33%
WER: 66.67%
WER: 150.00%
